In [7]:
import os
os.uname()

posix.uname_result(sysname='Linux', nodename='Acer-JC', release='5.15.133.1-microsoft-standard-WSL2', version='#1 SMP Thu Oct 5 21:02:42 UTC 2023', machine='x86_64')

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime

# Get the path of chromedriver which you have install

def parse_creation_date(datetime_str: str) -> datetime:
    # 6/28/24 11:34 AM => 6/28/2024 11:34 AM
    date_part, time_part, meridiem = datetime_str.split(' ')
    month, day, year = date_part.split('/')
    full_year = f"20{year}"
    new_datetime_str = f"{month}/{day}/{full_year} {time_part} {meridiem}"
    date_format = "%m/%d/%Y %I:%M %p"
    creation_date = datetime.strptime(new_datetime_str, date_format)
    return creation_date


def startBot(username, password, url, last_crawled_datetime):
    print('Start bot....')
    # path = "./chrome-win64/chrome.exe"
    # giving the path of chromedriver to selenium webdriver
    driver = webdriver.Remote(
        command_executor='http://localhost:4444/wd/hub',
        options=webdriver.ChromeOptions()
    )

    # opening the website  in chrome.
    print('Opening URL....')
    driver.get(url)
    try:
        assert "iExchangeWeb" in driver.title
    except:
        driver.quit()

    # find the id or name or class of
    # username by inspecting on username input
    driver.find_element(By.ID,
        "userName").send_keys(username)
     
    # find the password by inspecting on password input
    driver.find_element(By.ID,
        "password").send_keys(password)
    
    # click on submit
    driver.find_element(By.CSS_SELECTOR,
        "#login-box .btn-primary").click()

    # Locate the <tbody> tag where the table is at
    try:
        # WebDriverWait(driver, 10).until(EC.url_changes(url))
        tbody = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'tbody'))
        )
        mailbox_url = driver.current_url
        assert 'mailbox/inbox' in mailbox_url
        print('successful login to mailbox!')
    except:
        driver.quit() 


    try:        
        # Get all the rows that all contain a <tr> tag
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        print(f'rows = {len(rows)}')
    except:
        print('error at: Get all the rows that all contain a <tr> tag')
        driver.quit()

    # Out of all the rows, find the "un-crawled" rows
    new_rows=[]
    # try:
    for i, row in enumerate(rows):
        print(f'getting uncrawled row {i}....')
        try:
            creation_date_str = row.find_element(By.XPATH, 
                                                    "./td[11]").text
            print('found creation_date_str....')
        except:
            print('error at: creation_date_str = row.find_element(By.XPATH,"./td[11]").text')
            driver.quit()
        
        try:
            creation_date = parse_creation_date(creation_date_str)
            print('parsed creation date....')
        except:
            print('error at: parse_creation_date')
            driver.quit()
        if last_crawled_datetime >= creation_date:
            break
        new_rows.append(row)



    # in each un-crawled row, click on its "view" folder-like button and collect the raw X12-format EDI data. 
    for row in reversed(new_rows): # reversed, start processing from earliest non-crawled date.
        # click on View Button and direct to EDI item page. 
        try:
            print(row)
            # view_button = row.find_element(By.XPATH, 
            #                     "./td[14]/button[1]") # click the "view" button
            # view_button = row.find_element(By.CSS_SELECTOR, 
            #                                '[title="View"]')
            view_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View"]'))
            )
            view_button.click()
            
            assert 'item' in driver.current_url
            print('successfully accessed EDI item data page! ')
        except:
            print('error at: click on its "view" folder-like button')
            driver.quit()


        # Crawl raw X12-format EDI data 

        # click on view source code button
        try:
            view_source_data_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View Source Data"]'))
            )
            view_source_data_button.click()

            # check if "Source Data" title is present
            # Need to Switch to iFrame first! 
            # Find the iframe element by id
            iframe_locator = driver.find_element(By.ID, "contentFrame")
            # Switch to the iframe context
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it(iframe_locator)
            )
            source_data_text = driver.find_element(By.XPATH, '/html/body/h1').text
            assert source_data_text == 'Source Data'
            print('Source Data is Here!')

        except:
            print('error at: click on view_source_data_button')
            driver.quit()


        # select view mode "Raw"
        

        
        # get raw X12-format EDI data 

        
        
        try:    
            # Navigate back to the previous page
            driver.get(mailbox_url)

            # Wait until the URL changes back to the previous one (optional)
            WebDriverWait(driver, 10).until(EC.url_to_be(mailbox_url))
            assert driver.current_url == mailbox_url
            print(f'directed back to mailbox page {driver.current_url}! ')
        except:
            print('error at: back to mailbox page')
            driver.quit()
    
    driver.quit()
            



# Driver Code
# Enter below your login credentials
username = "***REMOVED***"
password = "***REMOVED***"

# URL of the login page of site
# which you want to automate login.
url = "https://www.iexchangeweb.com/ieweb/general/login"

# Get the "new data": last datetime of crawled data ~ newest
date_format = "%m/%d/%Y %I:%M %p"
last_crawled_datetime = datetime.strptime("6/28/2024 2:28 PM", date_format)

# Call the function
startBot(username, password, url, last_crawled_datetime)

Start bot....
Opening URL....
successful login to mailbox!
rows = 100
getting uncrawled row 0....
found creation_date_str....
parsed creation date....
getting uncrawled row 1....
found creation_date_str....
parsed creation date....
getting uncrawled row 2....
found creation_date_str....
parsed creation date....
getting uncrawled row 3....
found creation_date_str....
parsed creation date....
getting uncrawled row 4....
found creation_date_str....
parsed creation date....
getting uncrawled row 5....
found creation_date_str....
parsed creation date....
getting uncrawled row 6....
found creation_date_str....
parsed creation date....
getting uncrawled row 7....
found creation_date_str....
parsed creation date....
getting uncrawled row 8....
found creation_date_str....
parsed creation date....
getting uncrawled row 9....
found creation_date_str....
parsed creation date....
<selenium.webdriver.remote.webelement.WebElement (session="8dae0d908680b3a6a07cd23de9055840", element="f.85874BB72426C5A